In [ ]:
from MDAnalysis.analysis import distances
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline
import numpy as np
from numpy.linalg import norm
import scipy as sp

import seaborn as sns ; sns.set_style("white")
import pandas as pd

import warnings
# suppress some MDAnalysis warnings when writing PDB files
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [ ]:
#Do mpl ssettings
plt.rcParams['axes.linewidth']=0.8
plt.rcParams['axes.labelweight']='bold'
plt.rcParams['figure.max_open_warning'] = 100
plt.rcParams['axes.labelpad']=2
plt.rcParams['axes.formatter.useoffset']=False
plt.rcParams['font.weight']='bold'
plt.rcParams['xtick.major.pad']=1
plt.rcParams['ytick.major.pad']=1
plt.rcParams['xtick.labelsize']=8
plt.rcParams['ytick.labelsize']=8
plt.rcParams['lines.markersize']=8
plt.rcParams['axes.titlepad']=1
# # dark gray, dim gray ,  dark cyan  ; cyan
# gr='#a9a9a9' ; Gr='#696969' ; dc='#008b8b' ;  cy ='#00ffff' ; gn="#00ff00"
# Ex='#3232cd' ; In='red'     ; Ad='#ff00ff'

#DO seaborn settings
mysets = {'lines.linewidth': 1.0, 'lines.markersize': 10}                  
sns.set_context("paper", rc = mysets)  
plt.style.use('classic')

plot_kwds = {'alpha' : 1.0, 's' : 80, 'linewidths':0}

loading trajectory and topology 

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
#u = mda.Universe('2GIV_ZAFF_solv_mod.prmtop', '03_Prod_compressed.trr')
u = mda.Universe('coavppar_agoinv.prmtop', '04_Prod_1.nc','04_Prod_2.nc', '04_Prod_3.nc', 
                 '04_Prod_4.nc', '04_Prod_5.nc', '04_Prod_6.nc')
# u = mda.Universe('2ath_fixed.pdb')

In [ ]:
def get_dist_type1(u, mask_A, mask_B):
    dist_AB = []
    for ts in u.trajectory:
        a1 = u.select_atoms(mask_A)
        a2 = u.select_atoms(mask_B)
        resids1, resids2, dist = distances.dist(a1, a2, offset=0)  
        dist_AB.append(dist)
        
    dist_AB = np.asarray(dist_AB)
    dist_AB = np.reshape(dist_AB, (-1))
    return dist_AB

def get_dist_type2(u, mask_A, mask_B):
    dist_AB = []
    for ts in u.trajectory:
        a1 = u.select_atoms(mask_A)
        a2 = u.select_atoms(mask_B)
        cog1 = a1.center_of_geometry()
        cog2 = a2.center_of_geometry()
        
        # Calculate distances between cog1 and cog2
        dist = np.linalg.norm(cog1 - cog2)
        # resids1, resids2, dist = distances.dist(a1, a2, offset=0)  
        dist_AB.append(dist)
        
    dist_AB = np.asarray(dist_AB)
    dist_AB = np.reshape(dist_AB, (-1))
    return dist_AB

<!-- Getting distances as features FOR  antaAGONIST:     
H-BOND -> GLN 283 - SER 464  -> HE22 - O  
Salt-Bridge -> ARG288 - O = -> HH21 - O38       -->

In [ ]:
[print(i) for i in u.select_atoms("resname EKP")]
# cog = u.select_atoms("resid 135 and (name H)")
# cog.center_of_geometry()

In [ ]:
# [print(i) for i in u.select_atoms("resid 86 and (name CD1 CE1 CZ CE2 CD2 CG)")]
[print(i) for i in u.select_atoms("resid 83 and (name CD1 CE1 CZ CE2 CD2 CG)")]
[print(i) for i in u.select_atoms("resname EKP and (name C01 C02 C03 C04 C05 C06)")]

In [ ]:
# mask_A='resname EPK and (name N3)'
# mask_B='resid 269 and (name HH)'
# dist_1_hb = get_dist_type1(u, mask_A, mask_B)

# mask_A='resid 82 and (name HE22)'
# mask_B='resid 263 and (name O)'
# dist_2_hb = get_dist_type1(u, mask_A, mask_B)

mask_A='resid 83 and (name CD1 CE1 CZ CE2 CD2 CG)'
mask_B='resname EKP and (name C01 C02 C03 C04 C05 C06)'
dist_1_pipi = get_dist_type2(u, mask_A, mask_B)

# mask_A='resid 267 and (name HH)'
# mask_B='resname 3EA and (name O30)'
# dist_2_hb = get_dist_type1(u, mask_A, mask_B)

In [ ]:
print(dist_1_pipi.mean(), dist_1_pipi.std())
# print(dist_2_pipi.mean(), dist_2_pipi.std())
# print(dist_1_hb.mean(), dist_1_hb.std())
# print(dist_2_hb.mean(), dist_2_hb.std())

In [ ]:
# df = pd.DataFrame({'HIS243-PYR': dist_1_pipi, 'HIS243-BENZ':dist_2_pipi,
#                    'HB1':dist_1_hb, 'HB2': dist_2_hb})
# #df = pd.DataFrame({'Zn-N':dist_Zn_NHID_noise, 'Zn-S':dist_Zn_S3_noise})
# df

In [ ]:
fig = plt.figure(figsize=(14, 4), facecolor='w', edgecolor='k', dpi=200)
gs = GridSpec(1, 2, figure=fig)

ax1 = fig.add_subplot(gs[0])
ax1.scatter(np.arange(dist_1_pipi.shape[0]), dist_1_pipi)
ax1.set_xlabel('time (x10$^{2}$ ps)')
ax1.set_ylabel('PHE$_{287}$-Lig$_{benzene}$ ($\AA$)')
ax1.set_xlim(-100,3100)

# ax2 = fig.add_subplot(gs[1])
# ax2.scatter(np.arange(dist_1_hb.shape[0]), dist_1_hb)
# ax2.set_xlabel('time (x10$^{2}$ ps)')
# ax2.set_ylabel('ARG$_{288}$-Lig$_{O38}$ ($\AA$)')
# ax2.set_xlim(-100,3100)

# ax3 = fig.add_subplot(gs[2])
# ax3.scatter(np.arange(dist_2_hb.shape[0]), dist_2_hb)
# ax3.set_xlabel('time (x10$^{2}$ ps)')
# ax3.set_ylabel('GLN$_{283}$-SER$_{464}$ ($\AA$)')
# ax3.set_xlim(-100,2700)

plt.tight_layout()
plt.savefig("distances_MD_COFAC_ANTAGO.png")

Load cross-corellation data for conformational changes

In [ ]:
# d_cross=[]
# with open("cross-cor_WAGO.dat", 'r') as file:
#     for line in file:
#         d_cross.append(line.strip().split())

# X_cross= np.asarray(d_cross, dtype=float)

In [ ]:
# mask = np.zeros_like(X_cross)
# mask[np.triu_indices_from(mask)] = True

# # Set up the matplotlib figure
# f, ax = plt.subplots(figsize=(10, 6), dpi=100, facecolor="w")
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# # sns.heatmap(X_cross, mask=mask, cmap=cmap, vmax=.3,
# #             square=True, xticklabels=15, yticklabels=10,
# #             linewidths=.5, cbar_kws={'shrink': 0.8})

# sns.heatmap(X_cross, mask=mask, cmap=cmap, square=True, xticklabels=15, yticklabels=10, cbar_kws={'shrink': 1.0})
# # sns.heatmap(X_cross, cmap='viridis', square=True, cbar_kws={'shrink': 0.5})
# plt.xlabel("no. Residues")
# plt.ylabel("no. Residues")
# plt.xticks(rotation=50)
# # plt.savefig("heatmap_crosscorr.png")

Computing RMSF

In [ ]:
average = align.AverageStructure(u, u, select='protein and name CA',
                                 ref_frame=0).run()
ref = average.results.universe

aligner = align.AlignTraj(u, ref,
                          select='protein and name CA',
                          in_memory=True).run()

c_alphas = u.select_atoms('protein and name CA')
R = rms.RMSF(c_alphas).run()

In [ ]:
pd.DataFrame(c_alphas.resids, R.results.rmsf).to_csv("rmsf_cofac_antagonist.csv")

In [ ]:
fig = plt.figure(figsize=(6,4), dpi=100, facecolor="w")
plt.plot(c_alphas.resids, R.results.rmsf, '-o')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
# plt.axvspan(22, 86, zorder=0, alpha=0.2, color='orange', label='22-86')
# plt.axvspan(158,172, zorder=0, alpha=0.2, color='green', label='158-172')
# plt.axvspan(245, 245, zorder=0, alpha=0.2, color='blue', label='245')
# plt.axvspan(358, 358, zorder=0, alpha=0.2, color='purple', label='358')
# plt.axvspan(576, 576, zorder=0, alpha=0.2, color='red', label='576')
# plt.axvspan(0, 100, zorder=0, alpha=0.2, color='green', label='NMP')
#plt.legend()

plt.tight_layout()
plt.savefig("rmsf_COFAC_ANTAGO_plots.png")

Computing RMSD

In [ ]:
ref = u.trajectory[0]
ref

In [ ]:
ref = u.trajectory[0]
print(ref)
print(u.select_atoms("protein"))

In [ ]:
ref = mda.Universe('firstframe2.pdb') 
R = mda.analysis.rms.RMSD(u, ref,
           select="protein")
R.run()

In [ ]:
reference = mda.Universe('coavppar_agoinv.prmtop', '04_Prod_1.nc','04_Prod_2.nc', '04_Prod_3.nc', 
                 '04_Prod_4.nc', '04_Prod_5.nc', '04_Prod_6.nc')

In [ ]:
# ref = mda.Universe('firstframe2.pdb') 
ref = reference. 
R = mda.analysis.rms.RMSD(u, reference,
           select="backbone",             # superimpose on whole backbone of the whole protein
           groupselections=["resname EKP", "resid 83"])                                    # NMP
R.run()

rmsd = R.rmsd.T   # transpose makes it easier for plotting
time = rmsd[1]

In [ ]:
rmsd

In [ ]:
rmsd_all = pd.read_csv("rmsd_all.csv", sep="\t")
rmsd_ekp = pd.read_csv("rmsd_ekp.csv", sep="\t")
rmsd_phe = pd.read_csv("rmsd_phe83.csv", sep="\t")

In [ ]:
rmsd_1 = np.array([list(map(float, row[0].split())) for row in rmsd_all.to_numpy()])
rmsd_2 = np.array([list(map(float, row[0].split())) for row in rmsd_ekp.to_numpy()])
rmsd_3 = np.array([list(map(float, row[0].split())) for row in rmsd_phe.to_numpy()])

In [ ]:
rmsd_2

In [ ]:
pd.DataFrame({'time':rmsd_1[:,0], 'all':rmsd_1[:,1], 'ligand':rmsd_2[:,1], 'residues':rmsd_3[:,1]}).to_csv("rmsd_all_cofac_antago.csv")

In [ ]:
fig = plt.figure(figsize=(8,6), dpi=100, facecolor="w")
ax = fig.add_subplot(111)
ax.plot(rmsd_1[:,0], rmsd_1[:,1], 'k-',  label="all")
ax.plot(rmsd_1[:,0], rmsd_2[:,1], 'b--', label="Ligand")
ax.plot(rmsd_1[:,0], rmsd_3[:,1], '--', label="HIS$_{449}$ TYR$_{473}$", c='tab:green')
ax.legend(loc="best")
ax.set_xlabel("time (x10$^{2}$ ps)")
ax.set_ylabel(r"RMSD ($\AA$)")
fig.savefig("rmsd_COFAC_ANTAGO_plots.png")

Computing PCA on scaled data: mixing distances and cross-corellation

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.decomposition import PCA

# data = pd.DataFrame([dist_1_hb, dist_2_hb, dist_1_pipi, dist_2_pipi, rmsd[2], rmsd[3], rmsd[4]])
data = np.vstack([dist_1_pipi, rmsd_1[:,1], rmsd_2[:,1], rmsd_3[:,1]]).T
# data = np.vstack([dist_1_hb, dist_1_pipi, rmsd[2], rmsd[3], rmsd[4]]).T
df = pd.DataFrame(data)
# scaler =  StandardScaler()
X = df.to_numpy()
scaler =  MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = scaler.fit_transform(X)
X_scaled

In [ ]:
pca = PCA(n_components=0.90)
X_pca = pca.fit_transform(X_scaled)
pca_variance = pca.explained_variance_
print(pca_variance, pca.explained_variance_ratio_)
pca_variance
plt.figure(figsize=(8, 6))
plt.bar(range(len(pca_variance)), pca_variance, alpha=0.5, align='center', label='individual variance')
plt.legend()
plt.ylabel('Variance ratio')
plt.xlabel('Principal components')

In [ ]:
fig, axs= plt.subplots(1,2, figsize=(8,4), dpi=100, facecolor="w")

axs[0].scatter(X_pca[:,0], X_pca[:,1], edgecolor='black')
axs[0].set_ylabel('PC1', fontsize=14)
axs[0].set_xlabel('PC2', fontsize=14)
axs[0].tick_params(axis='both', labelsize=12)

axs[1].scatter(X_pca[:,0], X_pca[:,2], edgecolor='black')
axs[1].set_ylabel('PC1', fontsize=14)
axs[1].set_xlabel('PC3', fontsize=14)
axs[1].tick_params(axis='both', labelsize=12)
plt.tight_layout()
# axs[0]t.savefig("pca_hessfit.png")
# plt.savefig("PCA12.png")

In [ ]:
X_pca.shape

Clustering with KMEANs

In [ ]:
kmin = 2
kmax = 10
niter = 1000

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics as skmetrics
from matplotlib.ticker import MaxNLocator

In [ ]:
validation2 = []
for k in range(kmin, kmax + 1):  # Added +1 to include kmax
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X_pca)
    # kmeans.fit(X_scaled)
    clusters = kmeans.labels_
    centers = kmeans.cluster_centers_

    # psf = skmetrics.calinski_harabasz_score(X_scaled, clusters)
    # wss = kmeans.inertia_
    # DBI = skmetrics.davies_bouldin_score(X_scaled, clusters)
    # sil = skmetrics.silhouette_score(X_scaled, clusters)

    psf = skmetrics.calinski_harabasz_score(X_pca, clusters)
    wss = kmeans.inertia_
    DBI = skmetrics.davies_bouldin_score(X_pca, clusters)
    sil = skmetrics.silhouette_score(X_pca, clusters)

    validation2.append((k, wss, psf, DBI, sil))

validation2 = np.asarray(validation2)


In [ ]:
fig=plt.figure(figsize=(8,6), dpi= 196, facecolor='w', edgecolor='k')
plt.subplot(221)
plt.title('WSS',fontsize=10)
plt.plot(validation2[:,0], validation2[:,1], 'rs--', linewidth=1)
ax =fig.gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

plt.subplot(222)
plt.title('pSF',fontsize=10)
plt.plot(validation2[:,0], validation2[:,2], 'rs--', linewidth=1)
ax = fig.gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

plt.subplot(223)
plt.title('DBI',fontsize=10)
plt.plot(validation2[:,0], validation2[:,3], 'rs--', linewidth=1)
ax = fig.gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

plt.subplot(224)
plt.title('Silhouette score',fontsize=10)
plt.plot(validation2[:,0], validation2[:,4], 'rs--',linewidth=1)
ax = fig.gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

plt.tight_layout()

plt.savefig("validation_scores_PCA_MD_COFAC_ANTAGO.png", dpi=300)

five clusters centroids found !

In [ ]:
X_pca

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0, tol=1e-9, max_iter=1000)
kmeans.fit(X_pca)
# kmeans.fit(X_scaled)
clusters = kmeans.labels_
centers = kmeans.cluster_centers_

In [ ]:
X_pca.shape

In [ ]:
centers[:,0]

In [ ]:
print(X_pca[:10,0])
print(X_pca[:10,0].round(5))

In [ ]:
from scipy.spatial import distance
indices = []
for centroid in centers:
    closest_index = np.argmin([distance.euclidean(centroid, point) for point in X_pca])
    indices.append(closest_index)

In [ ]:
indices

In [ ]:
pd.Series(indices).to_csv("indices_centroids_frame_COFAC_ANTAGO.txt")

In [ ]:
# plt.figure(figsize=(10,8), dpi=196, facecolor="w")

# plt.scatter(np.arange(X_scaled.shape[0]), X_scaled[:,0], c=clusters, edgecolors='k')
# plt.scatter(indices, centers[:, 1], marker='x', s=200, color='red', label='Centroids')
# plt.scatter(indices, centers[:, 1], marker='x', s=200, color='red', label='Centroids')
# plt.xlabel('time (x10$^{2}$ ps)')
# plt.ylabel('HIS$_{243}$-Lig$_{pyrrole}$ scaled ($\AA$)')

In [ ]:
fig, axs= plt.subplots(1,2, figsize=(8,4), dpi=196, facecolor="w")

# axs[0].scatter(X_pca[:,0], X_pca[:,1], edgecolor='black')
axs[0].scatter(X_pca[:,0], X_pca[:,1], c=clusters, edgecolor='black')
axs[0].scatter(centers[:,0], centers[:,1], marker='x', s=200, color='red')
axs[0].set_ylabel('PC1', fontsize=14)
axs[0].set_xlabel('PC2', fontsize=14)
axs[0].tick_params(axis='both', labelsize=12)

# axs[1].scatter(X_pca[:,0], X_pca[:,2], edgecolor='black')
axs[1].scatter(X_pca[:,0], X_pca[:,2], c=clusters, edgecolor='black')
axs[1].scatter(centers[:,0], centers[:,2], marker='x', s=200, color='red')
axs[1].set_ylabel('PC1', fontsize=14)
axs[1].set_xlabel('PC3', fontsize=14)
axs[1].tick_params(axis='both', labelsize=12)

plt.tight_layout()
plt.savefig("PCA_clusters_MD_COFAC_ANTAGO.png")

 writing clister centroids as pdbs

In [ ]:
ag = u.select_atoms("not resname HOH WAT H2O")
with mda.Writer('centroids_COFAC_ANTAGO.pdb', ag.n_atoms) as w:
    for ts in u.trajectory[indices]:
        w.write(ag)
